In [1]:
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn import metrics, ensemble
import xgboost as xgb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
import sys
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

### Normalization

In [2]:
df1 = pd.read_csv('./data/melon100.csv', encoding='utf-8')

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 19 columns):
Filename               100 non-null object
ALBUM                  100 non-null object
ALBUMARTIST            64 non-null object
ARTIST                 100 non-null object
COMMENT                1 non-null object
COMPOSER               1 non-null object
COPYRIGHT              7 non-null object
DISCNUMBER             80 non-null float64
ENCODEDBY              7 non-null object
GENRE                  100 non-null object
LENGTH                 2 non-null float64
TITLE                  100 non-null object
TRACK                  99 non-null float64
UNSYNCEDLYRICS         98 non-null object
Upload                 1 non-null object
UPLOAD BY K2NBLOG      4 non-null object
UPLOADED BY K2NBLOG    4 non-null object
WWW                    11 non-null object
YEAR                   100 non-null object
dtypes: float64(3), object(16)
memory usage: 14.9+ KB


In [4]:
df1.head()

,Filename,ALBUM,ALBUMARTIST,ARTIST,COMMENT,COMPOSER,COPYRIGHT,DISCNUMBER,ENCODEDBY,GENRE,LENGTH,TITLE,TRACK,UNSYNCEDLYRICS,Upload,UPLOAD BY K2NBLOG,UPLOADED BY K2NBLOG,WWW,YEAR
0,001. 김나영 - 솔직하게 말해서 나.mp3,솔직하게 말해서 나,NaN,김나영,NaN,NaN,NaN,1.0,NaN,Ballad,NaN,솔직하게 말해서 나,1.0,eng||햇살이 좋았던 그 날\r\n그만하자며 말을 하는 너\r\n전혀 믿어지지가 ...,NaN,NaN,NaN,NaN,20190609
1,"002. 장혜진, 윤민수 - 술이 문제야.mp3",술이 문제야,"장혜진, 윤민수","장혜진, 윤민수",NaN,NaN,NaN,NaN,NaN,Ballad,NaN,술이 문제야 (Drunk On Love),1.0,eng||그리워서 한 잔\r\n생각나서 한 잔\r\n내 눈물 섞어 한 잔\r\n또 ...,NaN,NaN,NaN,NaN,2019.06.18
2,003. Anne-Marie - 2002.mp3,Speak Your Mind [Deluxe],Anne-Marie,Anne-Marie,NaN,NaN,NaN,1.0,NaN,"R&B, Pop",NaN,2002,10.0,kor||I will always remember\r\nThe day you kis...,NaN,NaN,NaN,NaN,20180427
3,004. 임재현 - 사랑에 연습이 있었다면.mp3,사랑에 연습이 있었다면,임재현,임재현,NaN,NaN,NaN,1.0,NaN,Ballad,NaN,사랑에 연습이 있었다면,1.0,kor||사랑에 연습이 있었다면\r\n우리는 달라졌을까\r\n내가 널 만난 시간 혹...,NaN,NaN,NaN,NaN,2018
4,005. 다비치 - 너에게 못했던 내 마지막 말은.mp3,너에게 못했던 내 마지막 말은,NaN,다비치,NaN,NaN,NaN,1.0,NaN,Ballad,NaN,너에게 못했던 내 마지막 말은,1.0,eng||아직 남았을까\r\n못 잊을 이유가 남아 있을까\r\n나만 왜 이렇게 \r...,NaN,NaN,NaN,NaN,20190517


In [5]:
del df1['ALBUMARTIST']
del df1['COMMENT']
del df1['COMPOSER']
del df1['COPYRIGHT']
del df1['DISCNUMBER']
del df1['ENCODEDBY']
del df1['LENGTH']
del df1['TRACK']
del df1['Upload']
del df1['UPLOAD BY K2NBLOG']
del df1['UPLOADED BY K2NBLOG']
del df1['WWW']
del df1['UNSYNCEDLYRICS']

In [6]:
df1 = df1.dropna()
#df1 = df1.fillna(value=0)

In [7]:
df1 = df1.reset_index(drop=True)

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
Filename    100 non-null object
ALBUM       100 non-null object
ARTIST      100 non-null object
GENRE       100 non-null object
TITLE       100 non-null object
YEAR        100 non-null object
dtypes: object(6)
memory usage: 4.8+ KB


In [9]:
for i in df1.Filename.index :
    df1.Filename[i] = df1.Filename[i][5:-4]
    
for i in df1.YEAR.index :    
    df1.YEAR[i] = df1.YEAR[i][:4]

In [10]:
df1

,Filename,ALBUM,ARTIST,GENRE,TITLE,YEAR
0,김나영 - 솔직하게 말해서 나,솔직하게 말해서 나,김나영,Ballad,솔직하게 말해서 나,2019
1,"장혜진, 윤민수 - 술이 문제야",술이 문제야,"장혜진, 윤민수",Ballad,술이 문제야 (Drunk On Love),2019
2,Anne-Marie - 2002,Speak Your Mind [Deluxe],Anne-Marie,"R&B, Pop",2002,2018
3,임재현 - 사랑에 연습이 있었다면,사랑에 연습이 있었다면,임재현,Ballad,사랑에 연습이 있었다면,2018
4,다비치 - 너에게 못했던 내 마지막 말은,너에게 못했던 내 마지막 말은,다비치,Ballad,너에게 못했던 내 마지막 말은,2019
5,어반자카파 - 서울 밤 (Feat. 빈지노),서울 밤,어반 자카파(Urban Zakapa),"R&B, Soul",서울 밤 (Feat. 빈지노),2019
6,Billie Eilish - bad guy,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",Billie Eilish,Alternative,bad guy,2019
7,방탄소년단 - 작은 것들을 위한 시 (Boy with Luv),MAP OF THE SOUL : PERSONA,"방탄소년단,Halsey","Rap, Hip",작은 것들을 위한 시 (Boy with Luv),2019
8,이하이 - 누구 없소 (NO ONE) (Feat. B.I of iKON),24℃,이하이,Dance,누구 없소 (NO ONE) (Feat. B.I of iKON),2019
9,Naomi Scott - Speechless (Full),Aladdin (Original Motion Picture Soundtrack),Naomi Scott,Soundtrack,Speechless (Full),2019


In [11]:
df1.to_csv('Melon_top_100_test.csv', encoding='ms949')

In [17]:
df2 = pd.read_csv('./data/Melon_top_100.csv', encoding='ms949')

In [18]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
Unnamed: 0    100 non-null int64
Filename      100 non-null object
ALBUM         100 non-null object
ARTIST        100 non-null object
GENRE         100 non-null object
TITLE         100 non-null object
YEAR          100 non-null int64
TAG           100 non-null object
dtypes: int64(2), object(6)
memory usage: 6.3+ KB


In [19]:
del df2['Unnamed: 0']

In [20]:
df2

,Filename,ALBUM,ARTIST,GENRE,TITLE,YEAR,TAG
0,김나영 - 솔직하게 말해서 나,솔직하게 말해서 나,김나영,Ballad,솔직하게 말해서 나,2019,#발라드 #국내 #애절한 #발라드한 #애절하고 슬픈 멜로디의 노래 #국내외 록 발라...
1,"장혜진, 윤민수 - 술이 문제야",술이 문제야,"장혜진, 윤민수",Ballad,술이 문제야 (Drunk On Love),2019,#발라드 #국내 #애절한 #발라드한 #애절하고 슬픈 멜로디의 노래 #국내외 록 발라...
2,Anne-Marie - 2002,Speak Your Mind [Deluxe],Anne-Marie,"R&B, Pop",2002,2018,#R&B #Soul #국내 #인디 #썸 탈때 #오후 #사랑/기쁨 #어쿠스틱 #카페 ...
3,임재현 - 사랑에 연습이 있었다면,사랑에 연습이 있었다면,임재현,Ballad,사랑에 연습이 있었다면,2018,#발라드 #국내 #애절한 #발라드한 #애절하고 슬픈 멜로디의 노래 #국내외 록 발라...
4,다비치 - 너에게 못했던 내 마지막 말은,너에게 못했던 내 마지막 말은,다비치,Ballad,너에게 못했던 내 마지막 말은,2019,#발라드 #국내 #애절한 #발라드한 #애절하고 슬픈 멜로디의 노래 #국내외 록 발라...
5,어반자카파 - 서울 밤 (Feat. 빈지노),서울 밤,어반 자카파(Urban Zakapa),"R&B, Soul",서울 밤 (Feat. 빈지노),2019,#R&B #Soul #국내 #인디 #썸 탈때 #오후 #사랑/기쁨 #어쿠스틱 #카페 ...
6,Billie Eilish - bad guy,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",Billie Eilish,Alternative,bad guy,2019,#국외 #오후 #어쿠스틱 #감성적인 #밤/새벽 #잔잔한 #몽환적인 #쓸쓸한 #어쿠스...
7,방탄소년단 - 작은 것들을 위한 시 (Boy with Luv),MAP OF THE SOUL : PERSONA,"방탄소년단,Halsey","Rap, Hip",작은 것들을 위한 시 (Boy with Luv),2019,#국내 #힙합 #랩 #듀엣/피처링 #고음질FLAC #달콤 쌉싸름한 멜로우 감성 힙합...
8,이하이 - 누구 없소 (NO ONE) (Feat. B.I of iKON),24℃,이하이,Dance,누구 없소 (NO ONE) (Feat. B.I of iKON),2019,#국내 #댄스 #썸 탈때 #오후 #사랑/기쁨 #어쿠스틱 #카페 #나른한 오후에 듣기...
9,Naomi Scott - Speechless (Full),Aladdin (Original Motion Picture Soundtrack),Naomi Scott,Soundtrack,Speechless (Full),2019,#국외 #오후 #어쿠스틱 #감성적인 #밤/새벽 #잔잔한 #몽환적인 #쓸쓸한 #어쿠스...


In [21]:
Tag_list = []
for a in df2.TAG :
    Tag_list.append(a)

In [23]:
Tag_list

['#발라드 #국내 #애절한 #발라드한 #애절하고 슬픈 멜로디의 노래 #국내외 록 발라드 명곡 콜렉션 #고음질FLAC',
 '#발라드 #국내 #애절한 #발라드한 #애절하고 슬픈 멜로디의 노래 #국내외 록 발라드 명곡 콜렉션 #고음질FLAC',
 '#R&B #Soul #국내 #인디 #썸 탈때 #오후 #사랑/기쁨 #어쿠스틱 #카페 #나른한 오후에 듣기좋은 노래 #썸 탈때 듣는 두근두근 노래 #사랑을 주제로 한 러브송 #어쿠스틱한 노래모음 #카페에서 듣기좋은 노래',
 '#발라드 #국내 #애절한 #발라드한 #애절하고 슬픈 멜로디의 노래 #국내외 록 발라드 명곡 콜렉션 #고음질FLAC',
 '#발라드 #국내 #애절한 #발라드한 #애절하고 슬픈 멜로디의 노래 #국내외 록 발라드 명곡 콜렉션 #고음질FLAC',
 '#R&B #Soul #국내 #인디 #썸 탈때 #오후 #사랑/기쁨 #어쿠스틱 #카페 #나른한 오후에 듣기좋은 노래 #썸 탈때 듣는 두근두근 노래 #사랑을 주제로 한 러브송 #어쿠스틱한 노래모음 #카페에서 듣기좋은 노래',
 '#국외 #오후 #어쿠스틱 #감성적인 #밤/새벽 #잔잔한 #몽환적인 #쓸쓸한 #어쿠스틱한 노래모음 #여름 밤의 진한 감성충전음악 #잠못 이룬 새벽에 듣기좋은 음악 #강력추천 센티멘탈 감성 팝 #편안하게 듣는 잔잔한 음악 #꿈을 꾸는 듯한 몽환적 사운드 #쓸쓸한 느낌을 주는 공허한 멜로디 # 쓸쓸한',
 '#국내 #힙합 #랩 #듀엣/피처링 #고음질FLAC #달콤 쌉싸름한 멜로우 감성 힙합 #피처링&콜라보가 돋보이는 음악',
 '#국내 #댄스 #썸 탈때 #오후 #사랑/기쁨 #어쿠스틱 #카페 #나른한 오후에 듣기좋은 노래 #썸 탈때 듣는 두근두근 노래 #사랑을 주제로 한 러브송 #어쿠스틱한 노래모음 #카페에서 듣기좋은 노래',
 '#국외 #오후 #어쿠스틱 #감성적인 #밤/새벽 #잔잔한 #몽환적인 #쓸쓸한 #어쿠스틱한 노래모음 #여름 밤의 진한 감성충전음악 #잠못 이룬 새벽에 듣기좋은 음악 #강력추천 센티멘탈 감성 팝 #편안하게 듣는 잔잔한 음악 #

In [24]:
df3 = pd.read_csv('./data/user.csv', encoding='euc-kr')

In [25]:
df3.head()

,USERID,AGE,SEX,SINGERS,GENRES,TAGS
0,1,14,0,"김나영, 폴킴, 레드벨벳","Ballad, R&B, Indie",#가요 #이별 #오후
1,2,13,1,"윤민수, 임재현, 엠씨더맥스","Pop, Dance, Trot",#록 # 인디 #잔잔한
2,3,12,1,"블랙핑크, 염따, 마크툽","Ballad, R&B, Indie",#듀엣 #어쿠스틱 #카페
3,4,11,0,"소유, 홍진영, 숀","Dance, Ballad, Rock",#썸 탈때 #잔잔한 #밤/새벽
4,5,10,0,"우디, 잔나비, 하은","Dance, Ballad, Rock",#악기 #카페 # 몽환적인


In [26]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
USERID     100 non-null int64
AGE        100 non-null int64
SEX        100 non-null int64
SINGERS    100 non-null object
GENRES     100 non-null object
TAGS       100 non-null object
dtypes: int64(3), object(3)
memory usage: 4.8+ KB
